In [ ]:
# Import the required libraries

import math
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa import tsatools, stattools
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots
from dmba import regressionSummary

%matplotlib inline


In [ ]:
# Code for creating a time series plot


# Read the data
Amtrak_df = pd.read_csv('Amtrak.csv')

# Convert the date information to a datetime object
Amtrak_df['Date'] = pd.to_datetime(Amtrak_df.Month, format='%d/%m/%Y')

# Convert dataframe column to series (Amtrak ridership series)
#ridership_ts = pd.Series(Amtrak_df.Ridership.values, index=Amtrak_df.Date)

# Create the ridership_ts dataframe with 'Ridership' as the column title
ridership_ts = pd.DataFrame(Amtrak_df.Ridership.values, index=Amtrak_df.Date, columns=["Ridership"])

# Define the time series frequency
ridership_ts.index = pd.date_range(start=ridership_ts.index[0], periods=len(ridership_ts), freq=ridership_ts.index.inferred_freq)

# Plot the series
ax = ridership_ts.plot()
ax.set_xlabel('Time')
ax.set_ylabel('Ridership (in 000s)')
ax.set_ylim(1300, 2600)
plt.show()


In [ ]:
print(ridership_ts)

In [ ]:
print(Amtrak_df)

In [ ]:


#ridership_df = tsatools.add_trend(ridership_ts, trend='ct')
#ridership_lm = sm.ols(formula='Ridership - trend', data=ridership_df).fit()

# Add a trend component to the DataFrame
ridership_df = sm.tsa.add_trend(ridership_ts, trend='ct')

# Define the linear model using the formula interface
ridership_lm = sm.OLS.from_formula('Ridership ~ trend', data=ridership_df).fit()

# Print the summary of the regression
print(ridership_lm.summary())

In [ ]:
# Plot the time series
ax = ridership_ts.plot()
ax.set_xlabel('Time')
ax.set_ylabel('Ridership (in 000s)')
ax.set_ylim(1300, 2300)

In [ ]:
# Plot the linear trend model
ridership_lm.predict(ridership_df).plot(ax=ax)
plt.show()

In [ ]:
# Plot the series and the fitted values
plt.figure(figsize=(10, 6))
plt.plot(ridership_ts.index, ridership_ts['Ridership'], label='Actual Ridership')
plt.plot(ridership_ts.index, ridership_lm.fittedvalues, label='Fitted Trend', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Ridership (in 000s)')
plt.ylim(1300, 2600)
plt.legend()
plt.show()

In [ ]:


# Fit linear model using training set and predict on validation set
#ridership_lm = sm.ols(formula='Ridership ~ trend', data=train_df).fit()
predict_df = ridership_lm.predict(valid_df)

# Create the plot
def singleGraphLayout(ax, ylim, train_df, valid_df):
    ax.set_xlim('1990', '2004-6')
    ax.set_ylim(*ylim)
    ax.set_xlabel('Time')
    ax.set_ylabel('Ridership (in 000s)')
    one_month = pd.Timedelta('31 days')
    xtrain = (min(train_df.index), max(train_df.index) - one_month)
    xvalid = (min(valid_df.index) - one_month, max(valid_df.index) - one_month)
    xtv = xtrain[1] + 0.5 * (xvalid[0] - xtrain[1])
    ypos = 0.9 * ylim[1]
    ax.add_line(plt.Line2D(xtrain, (ypos, ypos), color='black', linewidth=0.5))
    ax.add_line(plt.Line2D(xvalid, (ypos, ypos), color='black', linewidth=0.5))
    ax.axvline(x=xtv, ymin=0, ymax=1, color='black', linewidth=0.5)
    ax.text('1995', ypos + 50, 'Training')
    ax.text('2001-3', ypos + 50, 'Validation')

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))
train_df.plot(ax=axes[0], color='C0', linewidth=0.75)
valid_df.plot(ax=axes[0], color='C0', linestyle='dashed', linewidth=0.75)
ridership_lm.predict(train_df).plot(ax=axes[0], color='C1')
ridership_lm.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

singleGraphLayout(axes[0], (1300, 2600), train_df, valid_df)

residual = train_df['Ridership'] - ridership_lm.predict(train_df)
residual.plot(ax=axes[1], color='C1')
residual = valid_df['Ridership'] - ridership_lm.predict(valid_df)
residual.plot(ax=axes[1], color='C1', linestyle='dashed')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Forecast Errors')
axes[1].set_ylim(-800, 800)

plt.tight_layout()
plt.show()


In [ ]:
# Summary of output from a linear regression model applied to the Amtrak ridership data in the training period

ridership_lm.summary()

In [ ]:
# Code for creating Figure 17.3

# Predict linear ridership
ridership_lm_linear = smf.ols.from_formula('Ridership ~ trend', data=train_df).fit()

#ridership_lm_linear = sm.ols(formula='Ridership ~ trend', data=train_df).fit()
predict_df_linear = ridership_lm_linear.predict(valid_df)

# Predict exponential ridership
ridership_lm_expo = smf.ols(formula='Ridership ~ np.log(trend)', data=train_df).fit()
predict_df_expo = ridership_lm_expo.predict(valid_df)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 3.75))
train_df.plot(y='Ridership', ax=ax, color='C0', linewidth=0.75)
valid_df.plot(y='Ridership', ax=ax, color='C0', linestyle='dashed', linewidth=0.75)
singleGraphLayout(ax, [1300, 2600], train_df, valid_df)
ridership_lm_linear.predict(train_df).plot(ax=ax, color='C1')
predict_df_linear.plot(ax=ax, color='C1', linestyle='dashed')
ridership_lm_expo.predict(train_df).apply(lambda row: np.exp(row)).plot(ax=ax, color='C2')
predict_df_expo.apply(lambda row: np.exp(row)).plot(ax=ax, color='C2', linestyle='dashed')
ax.get_legend().remove()
plt.show()


In [ ]:
# Fit a polynomial regression model with trend and seasonality
ridership_lm_poly = sm.ols(formula='Ridership ~ trend + np.square(trend) + C(Month)', data=train_df).fit()

# Create the plot
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))

train_df.plot(y='Ridership', ax=axes[0], color='C0', linewidth=0.75)
valid_df.plot(y='Ridership', ax=axes[0], color='C0', linestyle='dashed', linewidth=0.75)

ridership_lm_poly.predict(train_df).plot(ax=axes[0], color='C1')
ridership_lm_poly.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

residual_train = train_df['Ridership'] - ridership_lm_poly.predict(train_df)
residual_train.plot(ax=axes[1], color='C1')

residual_valid = valid_df['Ridership'] - ridership_lm_poly.predict(valid_df)
residual_valid.plot(ax=axes[1], color='C1', linestyle='dashed')

graphLayout(axes, train_df, valid_df)
plt.show()


In [ ]:
# Summary of output from fitting trend and seasonality to Amtrak ridership in the training period

# Fit the model
ridership_lm_poly = sm.ols(formula='Ridership ~ trend + np.square(trend) + C(Month)', data=train_df).fit()

# Display the summary of the model
print(ridership_lm_poly.summary())


In [ ]:
# Fit AR(1) model on ridership residuals
formula = 'Ridership - trend + np.square(trend) + C(Month)'
train_lm_trendseason = sm.ols(formula=formula, data=train_df).fit()
train_res_arima = ARIMA(train_lm_trendseason.resid, order=(1, 0, 0),
                        freq='MS').fit(trend='nc')
forecast, _, conf_int = train_res_arima.forecast(1)

# ar1 = ARIMA(train_res, order=(1, 0, 0)).fit()

# Print coefficients of AR(1) model
print(pd.DataFrame({'coef': ar1.params}))

# Forecast residuals
forecast = ar1.forecast(steps=len(valid_df))
print(forecast)


In [ ]:
# Plotting the residuals and AR(1) model fit
ax = train_lm_trendseason.resid.plot(figsize=(9, 4))
train_res_arima = ar1.fittedvalues
train_res_arima.plot(ax=ax)
singleGraphLayout(ax, [-250, 250], train_df, valid_df)
plt.show()
